In [1]:
import pandas as pd

In [2]:
root_path = './dataset/'

In [3]:
comportamiento_df = pd.read_csv(f'{root_path}comportamiento_tarjetasvisa.csv', sep = ',')

In [4]:
informacion_df = pd.read_csv(f'{root_path}informacion_adicional_tarjetas.csv', sep = ',')